# Are we done yet?

Parsl and other concurrency libraries generally provide both **blocking** and **non-blocking** methods for accessing the results of asycnchronous method calls. By blocking, we are referring to methods that wait for the asynchronous opearation to complete before returning results, which blocks execution of the rest of a program. By non-blocking, we are referring to methods that return the result if it is available, but not if the async method hasn't completed yet. In that case, it lets the rest of the program to continue execution.

In practice this means that we can either 1) wait for all async calls to complete, and then process them using the blocking methods, or 2) query with a non-blocking method to see if each async call is complete, and only then retrieve the results for that method. We illustrate this approach below with parsl.

In [ ]:
# Required packages
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

# Configure the parsl executor
activate_env = 'workon scomp'
htex_local = Config(
    executors=[
        HighThroughputExecutor(
            max_workers_per_node=5,
            provider=LocalProvider(
                worker_init=activate_env
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

In [2]:
@python_app
def do_stuff(x):
    import time
    time.sleep(1)
    return x**2


In [4]:
import time
all_futures = []
for x in range(0,10):
    future = do_stuff(x)
    all_futures.append(future)
    print(future)

time.sleep(2)

for future in all_futures:
    print("Checking: ", future)
    if (future.done()):
        print("Do more with result: ", future.result())
    else:
        print("Sorry, come back later.")


<AppFuture at 0x7f628411c400 state=pending>
<AppFuture at 0x7f62943fdca0 state=pending>
<AppFuture at 0x7f628410b880 state=pending>
<AppFuture at 0x7f628410bd00 state=pending>
<AppFuture at 0x7f628410b430 state=pending>
<AppFuture at 0x7f628411f1f0 state=pending>
<AppFuture at 0x7f628411f5b0 state=pending>
<AppFuture at 0x7f628411f970 state=pending>
<AppFuture at 0x7f628411fd30 state=pending>
<AppFuture at 0x7f6284129130 state=pending>
Checking:  <AppFuture at 0x7f628411c400 state=finished returned int>
Do more with result:  0
Checking:  <AppFuture at 0x7f62943fdca0 state=finished returned int>
Do more with result:  1
Checking:  <AppFuture at 0x7f628410b880 state=finished returned int>
Do more with result:  4
Checking:  <AppFuture at 0x7f628410bd00 state=finished returned int>
Do more with result:  9
Checking:  <AppFuture at 0x7f628410b430 state=finished returned int>
Do more with result:  16
Checking:  <AppFuture at 0x7f628411f1f0 state=pending>
Sorry, come back later.
Checking:  <App